In [30]:
import pandas as pd
from sqlalchemy import create_engine
from config import sqlpw

### Load raw csvs into dataframes

In [31]:
# Load player.csv into dataframe
df_players_raw = pd.read_csv('./Resources/baseball/player.csv')

# Load college.csv into dataframe
df_college_raw = pd.read_csv('./Resources/baseball/college.csv')

# Load player_college.csv into dataframe
df_playercollege_raw = pd.read_csv('./Resources/baseball/player_college.csv')


### Explore loaded data and verify cleanliness for players.csv data

In [32]:
df_players_raw.head()

,player_id,birth_year,birth_month,birth_day,birth_country,birth_state,birth_city,death_year,death_month,death_day,...,name_last,name_given,weight,height,bats,throws,debut,final_game,retro_id,bbref_id
0,aardsda01,1981.0,12.0,27.0,USA,CO,Denver,NaN,NaN,NaN,...,Aardsma,David Allan,220.0,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01
1,aaronha01,1934.0,2.0,5.0,USA,AL,Mobile,NaN,NaN,NaN,...,Aaron,Henry Louis,180.0,72.0,R,R,1954-04-13,1976-10-03,aaroh101,aaronha01
2,aaronto01,1939.0,8.0,5.0,USA,AL,Mobile,1984.0,8.0,16.0,...,Aaron,Tommie Lee,190.0,75.0,R,R,1962-04-10,1971-09-26,aarot101,aaronto01
3,aasedo01,1954.0,9.0,8.0,USA,CA,Orange,NaN,NaN,NaN,...,Aase,Donald William,190.0,75.0,R,R,1977-07-26,1990-10-03,aased001,aasedo01
4,abadan01,1972.0,8.0,25.0,USA,FL,Palm Beach,NaN,NaN,NaN,...,Abad,Fausto Andres,184.0,73.0,L,L,2001-09-10,2006-04-13,abada001,abadan01


In [33]:
# Check for null or missing values in columns of interest
df_players_raw[['player_id', 'name_first', 'name_last']].isnull().sum()

# All players have a non-null player_id and name_last, but some players do not have a first name


player_id      0
name_first    39
name_last      0
dtype: int64

In [35]:
# Create df_player with non-null columns of interest
df_players = df_players_raw[['player_id', 'name_first', 'name_last']]
df_players = df_players.dropna() # Removes players who do not have a first name
df_players.head()

,player_id,name_first,name_last
0,aardsda01,David,Aardsma
1,aaronha01,Hank,Aaron
2,aaronto01,Tommie,Aaron
3,aasedo01,Don,Aase
4,abadan01,Andy,Abad


### Explore loaded data and verify cleanliness for college.csv data

In [36]:
df_college_raw.head()

,college_id,name_full,city,state,country
0,abilchrist,Abilene Christian University,Abilene,TX,USA
1,adelphi,Adelphi University,Garden City,NY,USA
2,adrianmi,Adrian College,Adrian,MI,USA
3,akron,University of Akron,Akron,OH,USA
4,alabama,University of Alabama,Tuscaloosa,AL,USA


In [37]:
# Determine if any fields have nulls or missing values
df_college_raw.isnull().sum()

college_id    0
name_full     0
city          0
state         0
country       0
dtype: int64

In [38]:
# No cleaning necessary
df_college = df_college_raw

### Explore loaded data and verify cleanliness for player_college.csv data

In [39]:
df_playercollege_raw.head()

,player_id,college_id,year
0,aardsda01,pennst,2001
1,aardsda01,rice,2002
2,aardsda01,rice,2003
3,abadan01,gamiddl,1992
4,abadan01,gamiddl,1993


In [40]:
# Determine if any fields have nulls or missing values
df_playercollege_raw.isnull().sum()

player_id     0
college_id    0
year          0
dtype: int64

In [41]:
# Data is listed by player-year (allowing data to reflect e.g. if a player transferred colleges).
# Extract only the single most-recently attended college for each player.

df_playercollege = df_playercollege_raw.groupby(['player_id']).max()
df_playercollege.head()


,college_id,year
player_id,,
aardsda01,rice,2003
abadan01,gamiddl,1993
abbeybe01,vermont,1892
abbotje01,kentucky,1994
abbotji01,michigan,1988


### Create cleaned df in preparation for writing to SQL DB

In [42]:
df = df_players.merge(df_playercollege, how='left', on='player_id').merge(df_college, how='left', on='college_id')
df.head()

,player_id,name_first,name_last,college_id,year,name_full,city,state,country
0,aardsda01,David,Aardsma,rice,2003.0,Rice University,Houston,TX,USA
1,aaronha01,Hank,Aaron,NaN,NaN,NaN,NaN,NaN,NaN
2,aaronto01,Tommie,Aaron,NaN,NaN,NaN,NaN,NaN,NaN
3,aasedo01,Don,Aase,NaN,NaN,NaN,NaN,NaN,NaN
4,abadan01,Andy,Abad,gamiddl,1993.0,Middle Georgia College,Cochran,GA,USA


In [43]:
df = df[['name_first', 'name_last', 'year', 'name_full', 'city', 'state', 'country']]
df = df.rename(columns={'year': 'grad_yr', 'name_full': 'college_name', 'city': 'college_city', 'state':'college_state', 'country': 'college_country'})
df.head()

,name_first,name_last,grad_yr,college_name,college_city,college_state,college_country
0,David,Aardsma,2003.0,Rice University,Houston,TX,USA
1,Hank,Aaron,NaN,NaN,NaN,NaN,NaN
2,Tommie,Aaron,NaN,NaN,NaN,NaN,NaN
3,Don,Aase,NaN,NaN,NaN,NaN,NaN
4,Andy,Abad,1993.0,Middle Georgia College,Cochran,GA,USA


### Connect to SQL DB and write cleaned df to SQL DB

In [44]:
# Connect to SQL DB
connection_string = f"root:{sqlpw}@localhost/etl_project"
engine = create_engine(f"mysql://{connection_string}") 

# Confirm tables
engine.table_names()

['colleges', 'players']

In [45]:
df.to_sql(name='players', con=engine, if_exists='replace', index=False)

In [47]:
pd.read_sql_query('select * from players', con=engine)

,name_first,name_last,grad_yr,college_name,college_city,college_state,college_country
0,David,Aardsma,2003.0,Rice University,Houston,TX,USA
1,Hank,Aaron,NaN,None,None,None,None
2,Tommie,Aaron,NaN,None,None,None,None
3,Don,Aase,NaN,None,None,None,None
4,Andy,Abad,1993.0,Middle Georgia College,Cochran,GA,USA
5,Fernando,Abad,NaN,None,None,None,None
6,John,Abadie,NaN,None,None,None,None
7,Ed,Abbaticchio,NaN,None,None,None,None
8,Bert,Abbey,1892.0,University of Vermont,Burlington,VT,USA
9,Charlie,Abbey,NaN,None,None,None,None
